In [ ]:
!pip install azure-iot-device
!pip install azure-iot-hub
!pip install python-dotenv

In [1]:
import sys
from dotenv import get_key, find_dotenv
sys.path.append('../utilities/video-analysis/shared/graph_manager')
from graph_manager import GraphManager

# find the environment file .env
env_path = find_dotenv(raise_error_if_not_found=True)

iotHubConnString = get_key(env_path, "IOT_HUB_CONN_STRING")
iotDeviceId = get_key(env_path, "IOT_DEVICE_ID")

moduleId = "lvaEdge" # Must be same as the name that we assigned to LVA module in the "deployment.lvaedge_rocketserver_rtspsim.template.json" file
operationsApiVersion = "1.0"  # Must be same as the version number in the "topology.json" file

graphManager = GraphManager(iotHubConnString, iotDeviceId, moduleId, operationsApiVersion)

### List existing topologies

In [2]:
# List topologies
response = graphManager.GenericCall("GraphTopologyList", {})
print(response)

{'additional_properties': {}, 'status': 200, 'payload': {'value': []}}


In [3]:
# List instances
response = graphManager.GenericCall("GraphInstanceList", {})
print(response)

{'additional_properties': {}, 'status': 200, 'payload': {'value': []}}


In [4]:
topologyFile = os.path.abspath("../modules/azureeyeh264module/lva-deployment/cvr-topology.json")
operationParams = {"topologyFile": topologyFile}

In [5]:
response = graphManager.GraphTopologySet(operationParams)
print(response)

{'additional_properties': {}, 'status': 201, 'payload': {'systemData': {'createdAt': '2020-09-28T20:15:30.156Z', 'lastModifiedAt': '2020-09-28T20:15:30.156Z'}, 'name': 'CVRToAMSAsset', 'properties': {'description': 'Continuous video recording to an Azure Media Services Asset', 'parameters': [{'name': 'rtspUserName', 'type': 'String', 'description': 'rtsp source user name.', 'default': 'dummyUserName'}, {'name': 'rtspPassword', 'type': 'String', 'description': 'rtsp source password.', 'default': 'dummyPassword'}, {'name': 'rtspUrl', 'type': 'String', 'description': 'rtsp Url'}], 'sources': [{'@type': '#Microsoft.Media.MediaGraphRtspSource', 'name': 'rtspSource', 'transport': 'Tcp', 'endpoint': {'@type': '#Microsoft.Media.MediaGraphUnsecuredEndpoint', 'url': '${rtspUrl}', 'credentials': {'@type': '#Microsoft.Media.MediaGraphUsernamePasswordCredentials', 'username': '${rtspUserName}'}}}], 'processors': [], 'sinks': [{'@type': '#Microsoft.Media.MediaGraphAssetSink', 'localMediaCachePath': 

### Set actual topology parameters

In [8]:
rtspUrl = get_key(env_path, "RTSP_URL")

topologyInstanceName =  "CVR-1"
topologyName = "CVRToAMSAsset"

mediaGraphTopologyParameters = {
          "name": topologyInstanceName,
          "properties": {
            "topologyName": topologyName,
            "description": "CVR Topology",
            "parameters": [
              {
                "name": "rtspUrl",
                "value": rtspUrl
              },
              {
                "name": "rtspUserName",
                "value": "username"
              },
              {
                "name": "rtspPassword",
                "value": "password"
              }
            ]
          }
        }

#### Set topology instance

In [9]:
# Set topology instance
response = graphManager.GenericCall("GraphInstanceSet", mediaGraphTopologyParameters)
print(response)

{'additional_properties': {}, 'status': 404, 'payload': {'error': {'code': 'NotFound', 'message': 'The requested resource was not found', 'details': [{'code': 'GraphTopologyNotFound', 'message': "Graph topology 'InferencingWithHttpExtension' not found"}]}}}


### Activate Graph

In [ ]:
# Activate topology instance
operationParams = {"name": topologyInstanceName}

response = graphManager.GenericCall("GraphInstanceActivate", operationParams)
print(response)

### Deactivate Graph

In [ ]:
graphManager.GenericCall("GraphInstanceDeactivate", operationParams)
print(response)



### Delete Graph

In [ ]:
graphManager.GenericCall("GraphInstanceDelete", operationParams)
print(response)

### Delete Topology

In [ ]:
deleteParams = {"name": topologyName}

graphManager.GenericCall("GraphTopologyDelete", deleteParams)
print(response)